In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


: 

In [ ]:
movie_data = pd.read_csv(r"C:\Users\parke\OneDrive\wiki_movie_plots_deduped.csv")
movie_data.info()

: 

In [3]:
# Select only films that are American in origin
us_movies = movie_data[movie_data['Origin/Ethnicity'] == 'American']

# Drop unwanted columns (Wiki.Page, Director, Cast)
us_movies = us_movies.drop(columns=['Wiki Page', 'Director', 'Cast'])

# Print the summary of the resulting DataFrame
us_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17377 entries, 0 to 17376
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      17377 non-null  int64 
 1   Title             17377 non-null  object
 2   Origin/Ethnicity  17377 non-null  object
 3   Genre             17377 non-null  object
 4   Plot              17377 non-null  object
dtypes: int64(1), object(4)
memory usage: 814.5+ KB


In [4]:
# Fetch Title and Plot columns
finaldata = us_movies[["Title", "Plot"]]
# Setting the movie title as index
finaldata = finaldata.set_index('Title')
finaldata.head()

,Plot
Title,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
The Martyred Presidents,"The film, just over a minute long, is composed..."
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
Jack and the Beanstalk,The earliest known adaptation of the classic f...


In [5]:
finaldata["Plot"] = finaldata["Plot"].astype(str)
print(finaldata.info())

<class 'pandas.core.frame.DataFrame'>
Index: 17377 entries, Kansas Saloon Smashers to Phantom Thread
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Plot    17377 non-null  object
dtypes: object(1)
memory usage: 271.5+ KB
None


In [9]:
#we will not need to process data to take out unimportant words

# Download stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\parke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
finaldata['Processed_Plot'] = finaldata['Plot'].apply(preprocess_text)

KeyboardInterrupt: 

: 

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 

# Vectorizing the pre-processed plot data using TF-IDF
tfidfvec = TfidfVectorizer() 
tfidf_plot = tfidfvec.fit_transform((finaldata["processedplot"])) 
  
# Find cosine similarity 
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = cosine_similarity(tfidf_plot, tfidf_plot)

# Storing the indices of the data 
indices = pd.Series(finaldata.index)

KeyError: 'processedplot'